In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [7]:
#!pip install torchvision

In [9]:
# Define the Generator with fewer layers and smaller output size (14x14)
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, 256),
            nn.ReLU(True),
            nn.Linear(256, output_dim),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

In [11]:
# Define Discriminator
# Define the Discriminator with smaller architecture
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [13]:
# Hyperparameters
latent_dim = 100
img_size = 14 * 14  # Reduced image size (14x14)
batch_size = 64
learning_rate = 0.0002
num_epochs = 20  # Fewer epochs for faster training

# Prepare the Data with image resizing to 14x14
transform = transforms.Compose([
    transforms.Resize(14),  # Resize images to 14x14
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

100%|██████████| 9.91M/9.91M [00:04<00:00, 2.47MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 65.6kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.06MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 2.28MB/s]


In [15]:
# Initialize models, loss function, and optimizers
generator = Generator(input_dim=latent_dim, output_dim=img_size)#.to('cuda')
discriminator = Discriminator(input_dim=img_size)#.to('cuda')

criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate)
optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate)

In [17]:
# Training Loop
for epoch in range(num_epochs):
    for i, (imgs, _) in enumerate(train_loader):
        
        # Adversarial ground truths
        valid = torch.ones((imgs.size(0), 1), requires_grad=False)#.to('cuda')
        fake = torch.zeros((imgs.size(0), 1), requires_grad=False)#.to('cuda')
        
        # Configure input
        real_imgs = imgs.view(imgs.size(0), -1)#.to('cuda')
        
        # Train Generator
        optimizer_G.zero_grad()
        z = torch.randn((imgs.size(0), latent_dim))#.to('cuda')
        gen_imgs = generator(z)
        g_loss = criterion(discriminator(gen_imgs), valid)
        g_loss.backward()
        optimizer_G.step()
        
        # Train Discriminator
        optimizer_D.zero_grad()
        real_loss = criterion(discriminator(real_imgs), valid)
        fake_loss = criterion(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        # Print progress
        if i % 100 == 0:
            print(f"Epoch [{epoch}/{num_epochs}] Batch {i}/{len(train_loader)} \
                  Loss D: {d_loss.item()}, loss G: {g_loss.item()}")

Epoch [0/20] Batch 0/938                   Loss D: 0.7103700637817383, loss G: 0.7121849060058594
Epoch [0/20] Batch 100/938                   Loss D: 0.5153858661651611, loss G: 0.7319075465202332
Epoch [0/20] Batch 200/938                   Loss D: 0.20059561729431152, loss G: 1.4934366941452026
Epoch [0/20] Batch 300/938                   Loss D: 0.07374270260334015, loss G: 2.4315104484558105
Epoch [0/20] Batch 400/938                   Loss D: 0.017348507419228554, loss G: 3.8128440380096436
Epoch [0/20] Batch 500/938                   Loss D: 0.05269809812307358, loss G: 3.093778610229492
Epoch [0/20] Batch 600/938                   Loss D: 0.0648210421204567, loss G: 2.5211353302001953
Epoch [0/20] Batch 700/938                   Loss D: 0.0678696408867836, loss G: 2.4523584842681885
Epoch [0/20] Batch 800/938                   Loss D: 0.12908178567886353, loss G: 3.4602043628692627
Epoch [0/20] Batch 900/938                   Loss D: 0.0965563952922821, loss G: 3.21447300910949

In [ ]:
# Visualizing Generated and Original Images
def show_images(gen_images, real_images):
    gen_images = gen_images.view(gen_images.size(0), 1, 14, 14).cpu().data
    real_images = real_images.view(real_images.size(0), 1, 14, 14).cpu().data
    
    # Concatenate generated and real images
    images = torch.cat([gen_images, real_images])
    
    grid = torchvision.utils.make_grid(images, nrow=8, normalize=True)
    
    # Set smaller figure size
    plt.figure(figsize=(6, 6))  # Smaller figure
    plt.imshow(grid.permute(1, 2, 0))
    plt.title('Top Half: Generated Images, Bottom Half: Original Images')
    plt.axis('off')
    plt.show()

# Generate images
z = torch.randn(16, latent_dim)#.to('cuda')  # Reduced to 16 for a smaller plot
gen_imgs = generator(z)

# Get a batch of real images
real_imgs, _ = next(iter(train_loader))
real_imgs = real_imgs[:16].view(16, -1)#.to('cuda')  # Reduced to 16 for a smaller plot

# Show generated vs original images
show_images(gen_imgs, real_imgs)